In [6]:
import numpy as np
import xarray as xr
random = np.random.RandomState(seed=0)

In [5]:
random

RandomState(MT19937) at 0x7FE9069F4640

In [7]:
n_sample, n_tile, nx, ny, n_feature = 5, 6, 12, 12, 2
size=(n_sample, n_tile, nx, ny, n_feature)

In [9]:
xr.DataArray(
            random.uniform(low=0, high=10, size=size),
            dims=["sample", "tile", "x", "y", "z"],
            coords=[range(size[i]) for i in range(len(size))],
        )

<xarray.DataArray (sample: 5, tile: 6, x: 12, y: 12, z: 2)>
array([[[[[6.45894113e+00, 4.37587211e+00],
          [8.91773001e+00, 9.63662761e+00],
          [3.83441519e+00, 7.91725038e+00],
          ...,
          [7.80529176e+00, 1.18274426e+00],
          [6.39921021e+00, 1.43353287e+00],
          [9.44668917e+00, 5.21848322e+00]],

         [[4.14661940e+00, 2.64555612e+00],
          [7.74233689e+00, 4.56150332e+00],
          [5.68433949e+00, 1.87898004e-01],
          ...,
          [1.28926298e+00, 3.15428351e+00],
          [3.63710771e+00, 5.70196770e+00],
          [4.38601513e+00, 9.88373838e+00]],

         [[1.02044811e+00, 2.08876756e+00],
          [1.61309518e+00, 6.53108325e+00],
          [2.53291603e+00, 4.66310773e+00],
          ...,
...
          ...,
          [1.46508191e+00, 1.36721807e+00],
          [8.85839863e+00, 1.42671272e-01],
          [6.79763879e+00, 8.04785339e+00]],

         [[7.17913038e+00, 1.44409263e+00],
          [6.40503996e+00, 1.23486772e+00],
          [4.62247278e+00, 6.54832523e+00],
          ...,
          [6.90017764e+00, 4.70643618e+00],
          [3.83858836e+00, 1.44060190e+00],
          [9.87500201e+00, 7.27201097e+00]],

         [[4.55069492e+00, 4.92917735e+00],
          [2.53842712e-03, 2.96075067e-01],
          [8.07576643e+00, 4.61696955e-01],
          ...,
          [3.63413875e+00, 4.85819898e+00],
          [5.26453255e+00, 7.93133290e+00],
          [6.15129710e+00, 6.92809647e-01]]]]])
Coordinates:
  * sample   (sample) int64 0 1 2 3 4
  * tile     (tile) int64 0 1 2 3 4 5
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9 10 11
  * y        (y) int64 0 1 2 3 4 5 6 7 8 9 10 11
  * z        (z) int64 0 1

In [10]:
import tensorflow as tf

2022-07-26 18:50:39.937758: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-26 18:50:39.942223: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-26 18:50:39.942245: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [30]:
n_records = 10
nt, grid,nz = 40, 10,2

def records():
    for _ in range(n_records):
        record = {
            "a": np.random.uniform(0,1,[ nt, grid, nz]),
            "f": np.random.uniform(0,1,[ nt, grid, nz]),
        }
        print(record.keys)
        yield record
        
tfdataset = tf.data.Dataset.from_generator(
    records,
    output_types=(tf.float32),
    output_shapes=(tf.TensorShape([nt,grid,nz])),
)


In [31]:
records

<function __main__.records()>

In [34]:
for _ in range(n_records):
    record = {
        "a": np.random.uniform(0,1,[ nt, grid, nz]),
        "f": np.random.uniform(0,1,[ nt, grid, nz]),
    }


In [37]:
np.shape(record["f"])

(40, 10, 2)

In [22]:
record = {
    "a": np.random.uniform([ nt, grid, nz]),
    "f": np.random.uniform([ nt, grid, nz]),
}

In [26]:
record

{'a': array([31.10933808,  7.7081584 ,  1.09996103]),
 'f': array([17.02310492,  4.71533128,  1.92979082])}

In [29]:
np.shape(np.random.uniform(0,1,[ nt, grid, nz]))

(40, 10, 2)

In [38]:
import dataclasses
import numpy as np
import tensorflow as tf
import xarray as xr
import fv3fit
from fv3fit._shared.config import TRAINING_FUNCTIONS, get_hyperparameter_class
from fv3fit.tfdataset import tfdataset_from_batches
import fv3fit
from typing import Any, Callable, Optional, Sequence, TextIO, Tuple
from fv3fit._shared.config import TRAINING_FUNCTIONS, get_hyperparameter_class
from fv3fit._shared.hyperparameters import Hyperparameters
from graphPredict import PytorchModel
from fv3fit.tfdataset import tfdataset_from_batches
import pytest
from typing import Callable, Sequence, Union
import xarray as xr


GENERAL_TRAINING_TYPES = [
    "graph",
]

# automatically test on every registered training class
@pytest.fixture(params=GENERAL_TRAINING_TYPES)
def model_type(request):
    return request.param

@dataclasses.dataclass
class TrainingResult:
    model: PytorchModel
    output_variables: Sequence[str]
    test_dataset: xr.Dataset
    hyperparameters: Hyperparameters


def train_identity_model(model_type, sample_func, hyperparameters=None):
    input_variables, output_variables, train_dataset = get_data(
        model_type, sample_func
    )
    if hyperparameters is None:
        cls = get_hyperparameter_class(model_type)
        hyperparameters = cls.init_testing(input_variables, output_variables)
    input_variables, output_variables, test_dataset = get_data(
        model_type, sample_func
    )
    train_tfdataset = tfdataset_from_batches([train_dataset for _ in range(10)])
    val_tfdataset = tfdataset_from_batches([test_dataset])
    train = fv3fit.get_training_function(model_type)
    model = train(hyperparameters, train_tfdataset, val_tfdataset)
    return TrainingResult(model, output_variables, test_dataset, hyperparameters)



@pytest.mark.slow
def test_train_default_model_on_identity(model_type: str, regtest):
    """
    The model with default configuration options can learn the identity function,
    using gaussian-sampled data around 0 with unit variance.
    """

    fv3fit.set_random_seed(1)
    # don't set n_feature too high for this, because of curse of dimensionality
    n_sample, grid, n_feature = 5, 864 , 2
    sample_func = get_uniform_sample_func(size=(n_sample, grid, n_feature))

    assert_can_learn_identity(
        model_type, sample_func=sample_func, max_rmse=0.2, regtest=regtest,
    )

def get_uniform_sample_func(size, low=0, high=1, seed=0):
    random = np.random.RandomState(seed=seed)

    def sample_func():
        return xr.DataArray(
            random.uniform(low=low, high=high, size=size),
            dims=["sample", "grid", "z"],
            coords=[range(size[i]) for i in range(len(size))],
        )

    return sample_func


def assert_can_learn_identity(
    model_type,
    sample_func: Callable[[], xr.DataArray],
    max_rmse: float,
    regtest: Optional[TextIO] = None,
):
    """
    Args:
        model_type: type of model to train
        hyperparameters: model configuration
        sample_func: function that returns example DataArrays for training and
            validation, should return different data on subsequent calls
        max_rmse: maximum permissible root mean squared error
        regtest: if given, write hash of output dataset to this file object
    """
    result = train_identity_model(model_type, sample_func=sample_func)
    out_dataset = result.model.predict(result.test_dataset)
    for name in result.output_variables:
        assert out_dataset[name].dims == result.test_dataset[name].dims
    rmse = (
        np.mean(
            [
                np.mean((out_dataset[name] - result.test_dataset[name]) ** 2)
                / np.std(result.test_dataset[name]) ** 2
                for name in result.output_variables
            ]
        )
        ** 0.5
    )
    assert rmse < max_rmse



def get_data() -> tf.data.Dataset:
    n_records = 10
    nt, grid,nz = 40, 10,2
    low=0
    high=1
    
    def records():
        for _ in range(n_records):
            record = {
                "a": np.random.uniform(low, high,[ nt, grid, nz]),
                "f": np.random.uniform(low, high,[ nt, grid, nz]),
            }
            yield record
        
    tfdataset = tf.data.Dataset.from_generator(
        records,
        output_types=(tf.float32),
        output_shapes=(tf.TensorShape([nt,grid,nz])),
    )
    return tfdataset

    # set random seed
    # prepare randomly-generated "identity function" data
    # train the model
    # get the outputs from the model
    # check the outputs are sufficiently accurate

ModuleNotFoundError: No module named 'graphPredict'

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import NNConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utilsMPGNNUnet import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=6

coarsenInd=1
g = pickle.load(open("UpdatedGraph_Neighbour10", 'rb'))
residual=0
num_step=15
aggregat='mean'

control_str='MultiSAGEUnet'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=50

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=20



lr=0.001
disablecuda ='store_true'
batch_size=1
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lead"
    + str(lead)
    + "_epochs_"
    + str(epochs)
    +"MP_Block_"
    +str(num_step)
    + "aggregat_"
    +aggregat
    +"coarsen_"
    +str(coarsenInd)
    +"residual_"
    +str(residual)
    +".pt"
)

print(savemodelpath)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(2):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(1,1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(1,1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(1,1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(1,1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(1,1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)
exteraVar=np.swapaxes(exteraVar,2, 1)
print(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")




edg=np.asarray(g.edges())
latInd=lat[edg[1]]
lonInd=lon[edg[1]]
latlon=[latInd.T,lonInd.T]
# latlon=np.swapaxes(latlon, 1, 0)
latlon=torch.from_numpy(np.swapaxes(latlon, 1, 0)).float()
latlon=latlon.to(device)


Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0

print('loading model')

class UnetGraphSAGE(nn.Module):
    def __init__(self, g, in_feats, h_feats,out_feat,num_step,aggregat):
        super(UnetGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,aggregat)
        self.conv2 = SAGEConv(h_feats, int(h_feats/2), aggregat)
        self.conv3 = SAGEConv(int(h_feats/2), int(h_feats/4), aggregat)
        self.conv4 = SAGEConv(int(h_feats/4), int(h_feats/4), aggregat)
        self.conv5 = SAGEConv(int(h_feats/2), int(h_feats/2), aggregat)
        self.conv6 = SAGEConv(h_feats, out_feat,aggregat)
        self.g=g
        self.num_step=num_step
        
    def forward(self, in_feat,exteraVar1):

        for _ in range(self.num_step):
            h = self.conv1(self.g, in_feat)
            h = F.relu(h)
            h = self.conv2(self.g, h)
            h = F.relu(h)
            h = self.conv3(self.g, h)
            h = F.relu(h)
            tuple = (self.conv4(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            tuple = (self.conv5(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            h = self.conv6(self.g, h)
            in_feat=torch.cat((h, torch.squeeze(exteraVar1)), 1).float()
        return h
        

loss = nn.MSELoss()
g = g.to(device)
model = UnetGraphSAGE(g,7,256, 2,num_step,aggregat).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))


for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-2*lead,:]
        y_train=train[:,lead:-lead,:]
        y_train2=train[:,2*lead::,:]

        x_val=val[:,0:-lead,:]
        y_val=val[:,lead::,:]

        if residual==1:
            y_train=y_train-x_train
            y_val=y_val-x_val


        x_train=np.swapaxes(x_train, 1, 0)
        y_train=np.swapaxes(y_train, 1, 0)
        y_train2=np.swapaxes(y_train2, 1, 0)

        x_train=np.swapaxes(x_train, 2, 1)
        y_train=np.swapaxes(y_train, 2, 1)
        y_train2=np.swapaxes(y_train2, 2, 1)

        x_train=torch.Tensor(x_train).to(device)
        y_train=torch.Tensor(y_train).to(device)
        y_train2=torch.Tensor(y_train2).to(device)

        train_data = torch.utils.data.TensorDataset(x_train, y_train, y_train2)
        train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)



        x_val=np.swapaxes(x_val, 1, 0)
        y_val=np.swapaxes(y_val, 1, 0)
        x_val=np.swapaxes(x_val, 2, 1)
        y_val=np.swapaxes(y_val, 2, 1)
        x_val=torch.Tensor(x_val).to(device)
        y_val=torch.Tensor(y_val).to(device)


        val_data = torch.utils.data.TensorDataset(x_val, y_val)
        val_iter = torch.utils.data.DataLoader(val_data, batch_size)
        

        if valInde==0:
            min_val_loss = np.inf
            valInde+=1

        l_sum, n = 0.0, 0
        for x, y, y2 in train_iter:

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import SAGEConv
from dgl.nn.pytorch import NNConv
import numpy as np
import dask.diagnostics
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import networkx as nx
import os
from torch.utils.data import DataLoader
import torch.optim as optim
import warnings
import time
import select as sl
import pickle
from load_data import *
from utilsMPGNNUnet import *
import wandb
from fv3net.artifacts.resolve_url import resolve_url
from vcm import get_fs

lead=6

coarsenInd=1
g = pickle.load(open("UpdatedGraph_Neighbour10", 'rb'))
residual=0
num_step=15
aggregat='mean'

control_str='MultiSAGEUnet'#'TNSTTNST' #'TNTSTNTST'

print(control_str)

epochs=50

variableList=['h500','h200','h850']
TotalSamples=8500
Chuncksize=20



lr=0.001
disablecuda ='store_true'
batch_size=1
drop_prob = 0
out_feat=2

savemodelpath = (
    "weight_layer_"
    + control_str
    + "_lead"
    + str(lead)
    + "_epochs_"
    + str(epochs)
    +"MP_Block_"
    +str(num_step)
    + "aggregat_"
    +aggregat
    +"coarsen_"
    +str(coarsenInd)
    +"residual_"
    +str(residual)
    +".pt"
)

print(savemodelpath)

BUCKET = "vcm-ml-experiments"
PROJECT = "full-model-emulation"

model_out_url = resolve_url(BUCKET, PROJECT, savemodelpath)
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/"


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_url = "gs://vcm-ml-scratch/ebrahimn/2022-07-02/experiment-1-y/fv3gfs_run/" 
state_training_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "atmos_dt_atmos.zarr")), consolidated=True
)
# state_training_data2 = xr.open_zarr(fsspec.get_mapper(os.path.join(data_url, 'sfc_dt_atmos.zarr')))
lat_lon_data = xr.open_zarr(
    fsspec.get_mapper(os.path.join(data_url, "state_after_timestep.zarr"))
)

landSea = xr.open_zarr(
    fsspec.get_mapper(
        "gs://vcm-ml-experiments/default/2022-05-09/baseline-35day-spec-sst/fv3gfs_run/state_after_timestep.zarr"
    )
)
landSea_Mask = landSea.land_sea_mask[1].load()
landSea_Mask = landSea_Mask[:, ::coarsenInd, ::coarsenInd].values.flatten()


lat=(lat_lon_data.latitude[1].load())
lon=(lat_lon_data.longitude[1].load())
lat=lat[:,::coarsenInd,::coarsenInd].values.flatten()
lon=lon[:,::coarsenInd,::coarsenInd].values.flatten()
# cosLat=np.expand_dims(np.cos(lat),axis=1)
# cosLon=np.expand_dims(np.cos(lon),axis=1)
# sinLat=np.expand_dims(np.sin(lat),axis=1)
# sinLon=np.expand_dims(np.sin(lon),axis=1)
cosLat=np.cos(lat)
cosLon=np.cos(lon)
sinLat=np.sin(lat)
sinLon=np.sin(lon)
for i in range(2):
        if i==0:
            sinLon=torch.tensor(sinLon).unsqueeze(0).repeat(1,1)
            cosLon=torch.tensor(cosLon).unsqueeze(0).repeat(1,1)
            sinLat=torch.tensor(sinLat).unsqueeze(0).repeat(1,1)
            cosLat=torch.tensor(cosLat).unsqueeze(0).repeat(1,1)
            landSea_Mask=torch.tensor(landSea_Mask).unsqueeze(0).repeat(1,1)
        elif i==1:
            sinLon=(sinLon).unsqueeze(0).repeat(batch_size,1,1)
            cosLon=(cosLon).unsqueeze(0).repeat(batch_size,1,1)
            sinLat=(sinLat).unsqueeze(0).repeat(batch_size,1,1)
            cosLat=(cosLat).unsqueeze(0).repeat(batch_size,1,1)
            landSea_Mask=(landSea_Mask).unsqueeze(0).repeat(batch_size,1,1)

exteraVar=torch.cat((sinLon, sinLat,cosLon,cosLat,landSea_Mask), 1).to(device)
exteraVar=np.swapaxes(exteraVar,2, 1)
print(device)

num_nodes=len(lon)
print(f"numebr of grids: {num_nodes}")




edg=np.asarray(g.edges())
latInd=lat[edg[1]]
lonInd=lon[edg[1]]
latlon=[latInd.T,lonInd.T]
# latlon=np.swapaxes(latlon, 1, 0)
latlon=torch.from_numpy(np.swapaxes(latlon, 1, 0)).float()
latlon=latlon.to(device)


Zmean=5765.8457   #Z500mean=5765.8457, 
Zstd=90.79599   #Z500std=90.79599

Tmean=10643.382          #Thickmean=10643.382
Tstd=162.12427              #Thickstd=162.12427
valInde=0

print('loading model')

class UnetGraphSAGE(nn.Module):
    def __init__(self, g, in_feats, h_feats,out_feat,num_step,aggregat):
        super(UnetGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats,aggregat)
        self.conv2 = SAGEConv(h_feats, int(h_feats/2), aggregat)
        self.conv3 = SAGEConv(int(h_feats/2), int(h_feats/4), aggregat)
        self.conv4 = SAGEConv(int(h_feats/4), int(h_feats/4), aggregat)
        self.conv5 = SAGEConv(int(h_feats/2), int(h_feats/2), aggregat)
        self.conv6 = SAGEConv(h_feats, out_feat,aggregat)
        self.g=g
        self.num_step=num_step
        
    def forward(self, in_feat,exteraVar1):

        for _ in range(self.num_step):
            h = self.conv1(self.g, in_feat)
            h = F.relu(h)
            h = self.conv2(self.g, h)
            h = F.relu(h)
            h = self.conv3(self.g, h)
            h = F.relu(h)
            tuple = (self.conv4(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            tuple = (self.conv5(self.g, h),h)
            h = torch.cat(tuple,dim=1)
            h = F.relu(h)
            h = self.conv6(self.g, h)
            in_feat=torch.cat((h, torch.squeeze(exteraVar1)), 1).float()
        return h
        

loss = nn.MSELoss()
g = g.to(device)
model = UnetGraphSAGE(g,7,256, 2,num_step,aggregat).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)
model.train()

all_indices=np.random.permutation(np.arange(start=0, stop=int(TotalSamples/Chuncksize)))


for epoch in range(1, epochs + 1):
    
    for ss in all_indices:

        Z500train=state_training_data[variableList[0]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain1=state_training_data[variableList[1]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()
        T2mtrain2=state_training_data[variableList[2]].isel(time=slice((ss*Chuncksize),(ss+1)*Chuncksize)).coarsen(grid_yt=coarsenInd).mean().coarsen(grid_xt=coarsenInd).mean()

        Z500train=np.swapaxes(Z500train.values, 1, 0)
        T2mtrain1=np.swapaxes(T2mtrain1.values, 1, 0)
        T2mtrain2=np.swapaxes(T2mtrain2.values, 1, 0)

        T2mtrain=T2mtrain1-T2mtrain2

        T2mtrain=T2mtrain.reshape(np.size(T2mtrain, 0), np.size(T2mtrain, 1)*np.size(T2mtrain, 2)*np.size(T2mtrain, 3))
        Z500train=Z500train.reshape(np.size(Z500train, 0), np.size(Z500train, 1)*np.size(Z500train, 2)*np.size(Z500train, 3))

        # Zmean = np.mean(Z500train)
        # Zstd = np.std(Z500train)

        # Tmean = np.mean(T2mtrain)
        # Tstd = np.std(T2mtrain)


        T2mtrain=(T2mtrain-Tmean)/Tstd
        Z500train=(Z500train-Zmean)/Zstd

        

        T2mtrain=np.expand_dims(T2mtrain,axis=0)
        Z500train=np.expand_dims(Z500train,axis=0)

        dataSets=np.concatenate((Z500train,T2mtrain),axis=0)

        num_samples=np.size(dataSets,1)
        print(f"Total samples: {num_samples}")


        len_val = round(num_samples * 0.25)
        len_train = round(num_samples * 0.75)
        train = dataSets[:,: len_train]
        val = dataSets[:,len_train+14: len_train + len_val]

        x_train=train[:,0:-2*lead,:]
        y_train=train[:,lead:-lead,:]
        y_train2=train[:,2*lead::,:]
        x_train[:,6,:]-y_train[:,0,:]

/home/ebrahimn/miniconda3/envs/dlwp2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'load_data'